# Single Frame Model

This script trains a single frame model based on an average image taken across the the forcasted month.

In [39]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, Conv3D, MaxPooling2D, MaxPooling3D
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam
import time
import numpy as np

## 1. Load Data

To begin, data for a single frame mnodel must be loaded along with the expected regional rainfall values.

### 1.0 New Training Data

This section loads the new training data set.

In [40]:
training_datafile = "D:/PHD_DATA/Video_18-01-2021/prepared-data/single_all.npy"
training_rainfallfile = "D:/PHD_DATA/Video_18-01-2021/prepared-data/expected_all.npy"

In [41]:
training_videos = np.load(training_datafile)
training_rainfall = np.load(training_rainfallfile)[:, 2:]

In [42]:
training_videos = np.swapaxes(training_videos, 1, 2)
training_videos = np.swapaxes(training_videos, 2, 3)

In [43]:
training_videos[:, :, :, 0] = (training_videos[:, :, :, 0] - np.min(training_videos[:, :, :, 0])) / (np.max(training_videos[:, :, :, 0]) - np.min(training_videos[:, :, :, 0]))
training_videos[:, :, :, 1] = (training_videos[:, :, :, 1] - np.min(training_videos[:, :, :, 1])) / (np.max(training_videos[:, :, :, 1]) - np.min(training_videos[:, :, :, 1]))

### 1.1 DEPRECATED

This following section uses two separate datasets. Which is not required.

In [44]:
#training_datafile = "E:/31-12-2020/prepared-data/single_train.npy"
#validation_datafile = "E:/31-12-2020/prepared-data/single_valid.npy"
#
#training_rainfallfile = "E:/31-12-2020/prepared-data/expected_train.npy"
#validation_rainfallfile = "E:/31-12-2020/prepared-data/expected_valid.npy"

In [45]:
#training_videos = np.load(training_datafile)
#validation_videos = np.load(validation_datafile)
#
#training_rainfall = np.load(training_rainfallfile)[:, 2:]
#validation_rainfall = np.load(validation_rainfallfile)[:, 2:]

In [46]:
# We need them in X, Y, COLOURS
#training_videos = np.swapaxes(training_videos, 1, 2)
#training_videos = np.swapaxes(training_videos, 2, 3)
#
#validation_videos = np.swapaxes(validation_videos, 1, 2)
#validation_videos = np.swapaxes(validation_videos, 2, 3)

In [47]:
# Scale the data
#training_videos[:, :, :, 0] = (training_videos[:, :, :, 0] - np.min(training_videos[:, :, :, 0])) / (np.max(training_videos[:, :, :, 0]) - np.min(training_videos[:, :, :, 0]))
#training_videos[:, :, :, 1] = (training_videos[:, :, :, 1] - np.min(training_videos[:, :, :, 1])) / (np.max(training_videos[:, :, :, 1]) - np.min(training_videos[:, :, :, 1]))
#
#validation_videos[:, :, :, 0] = (validation_videos[:, :, :, 0] - np.min(validation_videos[:, :, :, 0])) / (np.max(validation_videos[:, :, :, 0]) - np.min(validation_videos[:, :, :, 0]))
#validation_videos[:, :, :, 1] = (validation_videos[:, :, :, 1] - np.min(validation_videos[:, :, :, 1])) / (np.max(validation_videos[:, :, :, 1]) - np.min(validation_videos[:, :, :, 1]))

## 2. Model Definition

Next, a CNN model architecture is defined.

In [65]:
def model_generator(input_shape=(2, 61, 121), learning_rate=0.1):
    """ This method generates a model definition. """
    model = Sequential()
    
    # First layer
    model.add(Conv2D(32, (2, 2), input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Second layer
    model.add(Conv2D(32, (2, 2), input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Third layer
    model.add(Conv2D(16, (2, 2), input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Final Layer
    model.add(Flatten())
    model.add(Dense(13))
    model.add(Activation('relu'))
    
    # Setup training mechanism
    model.compile(
        loss="mean_squared_error",
        optimizer=Adam(learning_rate=learning_rate))
    
    return model

## 3. Model Training

Finally, training the model using the single framed data and opening a tensorboard instance with details.

In [66]:
def train_model(run_name, tensorboard, model, xdata, ydata, models_folder="D:/PHD_DATA/Video_18-01-2021/models/"):
    """ Trains the given model with the given dataset. """
    history = model.fit(
        xdata,
        ydata,
        batch_size=2,
        validation_split=0.3,
        callbacks=[tensorboard],
        epochs=50
    )
    save_model(model, models_folder + run_name + ".mdl")
    return history

In [67]:
# Flexible parameters
learning_rate = 0.10

In [68]:
# Run each model multiple times
for i in range(0, 5):
    run_name = "SL_32_32_16__13__22_t-{}".format(int(time.time()))
    tb = TensorBoard(log_dir=".\\logs\\{}".format(run_name))
    model = model_generator(learning_rate=learning_rate, input_shape=training_videos.shape[1:])
    history = train_model(run_name, tb, model, training_videos, training_rainfall)
    print("Final loss: {}".format(history.history["val_loss"][-1]))

Train on 191 samples, validate on 83 samples
Epoch 1/50
191/191 [==============================] - 4s 20ms/sample - loss: 10354.0891 - val_loss: 8203.5067
Epoch 2/50
191/191 [==============================] - 1s 8ms/sample - loss: 7465.1589 - val_loss: 8195.9841
Epoch 3/50
191/191 [==============================] - 1s 7ms/sample - loss: 7512.1671 - val_loss: 8170.0028
Epoch 4/50
191/191 [==============================] - 1s 6ms/sample - loss: 7479.2583 - val_loss: 8178.2882
Epoch 5/50
191/191 [==============================] - 2s 8ms/sample - loss: 7466.9084 - val_loss: 8195.9589
Epoch 6/50
191/191 [==============================] - 2s 10ms/sample - loss: 7455.1478 - val_loss: 8207.5854
Epoch 7/50
191/191 [==============================] - 1s 7ms/sample - loss: 7490.3007 - val_loss: 8318.3402
Epoch 8/50
191/191 [==============================] - 1s 7ms/sample - loss: 7461.3991 - val_loss: 8152.4799
Epoch 9/50
191/191 [==============================] - 1s 7ms/sample - loss: 7462.9735 - 

191/191 [==============================] - 1s 7ms/sample - loss: 9146.1049 - val_loss: 9804.5108
Epoch 26/50
191/191 [==============================] - 1s 6ms/sample - loss: 9144.6230 - val_loss: 9804.5624
Epoch 27/50
191/191 [==============================] - 1s 7ms/sample - loss: 9147.2563 - val_loss: 9788.8181
Epoch 28/50
191/191 [==============================] - 1s 6ms/sample - loss: 9135.3543 - val_loss: 9838.6959
Epoch 29/50
191/191 [==============================] - 1s 8ms/sample - loss: 9128.1961 - val_loss: 9826.2208
Epoch 30/50
191/191 [==============================] - 1s 7ms/sample - loss: 9150.9038 - val_loss: 9876.8837
Epoch 31/50
191/191 [==============================] - 1s 7ms/sample - loss: 9124.2937 - val_loss: 9998.2112
Epoch 32/50
191/191 [==============================] - 1s 6ms/sample - loss: 9163.9402 - val_loss: 9803.7552
Epoch 33/50
191/191 [==============================] - 1s 7ms/sample - loss: 9119.1081 - val_loss: 9790.0543
Epoch 34/50
191/191 [==========

191/191 [==============================] - 1s 7ms/sample - loss: 4037.3620 - val_loss: 4324.5975
Epoch 50/50
191/191 [==============================] - 1s 7ms/sample - loss: 4023.0857 - val_loss: 4634.0954
Final loss: 4634.095382506589
Train on 191 samples, validate on 83 samples
Epoch 1/50
191/191 [==============================] - 4s 23ms/sample - loss: 8683.9550 - val_loss: 8635.5147
Epoch 2/50
191/191 [==============================] - 2s 11ms/sample - loss: 7413.6231 - val_loss: 7818.3494
Epoch 3/50
191/191 [==============================] - 2s 10ms/sample - loss: 7363.2742 - val_loss: 7820.2926
Epoch 4/50
191/191 [==============================] - 2s 10ms/sample - loss: 7345.8133 - val_loss: 7839.0308
Epoch 5/50
191/191 [==============================] - 2s 10ms/sample - loss: 7361.5539 - val_loss: 7828.5604
Epoch 6/50
191/191 [==============================] - 1s 7ms/sample - loss: 7361.2779 - val_loss: 7930.0333
Epoch 7/50
191/191 [==============================] - 1s 6ms/sampl

191/191 [==============================] - 1s 7ms/sample - loss: 3654.0869 - val_loss: 3975.2968
Epoch 24/50
191/191 [==============================] - 1s 6ms/sample - loss: 3653.6522 - val_loss: 4002.8983
Epoch 25/50
191/191 [==============================] - 1s 7ms/sample - loss: 3691.2251 - val_loss: 3914.1380
Epoch 26/50
191/191 [==============================] - 1s 7ms/sample - loss: 3749.9836 - val_loss: 4471.4518
Epoch 27/50
191/191 [==============================] - 1s 7ms/sample - loss: 3752.8820 - val_loss: 3909.6348
Epoch 28/50
191/191 [==============================] - 1s 7ms/sample - loss: 3652.4263 - val_loss: 3935.6029
Epoch 29/50
191/191 [==============================] - 1s 7ms/sample - loss: 3778.9924 - val_loss: 4052.4305
Epoch 30/50
191/191 [==============================] - 1s 7ms/sample - loss: 3637.4054 - val_loss: 4049.1616
Epoch 31/50
191/191 [==============================] - 1s 7ms/sample - loss: 3741.3594 - val_loss: 3898.3902
Epoch 32/50
191/191 [==========